In [7]:
from datasets import load_dataset, load_metric
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np


In [2]:
ds = load_dataset("imdb")


Found cached dataset imdb (/home/prasann/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.96it/s]


In [3]:

model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2023-09-03 18:47:31,420] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def tokenize(examples):
    outputs = tokenizer(examples['text'], truncation=True)
    return outputs

tokenized_ds = ds.map(tokenize, batched=True)

In [8]:
def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
training_args = TrainingArguments(num_train_epochs=1,
          output_dir="distilbert-imdb",
          push_to_hub=False,
          per_device_train_batch_size=16,
          per_device_eval_batch_size=16,
          evaluation_strategy="epoch",
          logging_steps=100
        )

In [24]:
data_collator = DataCollatorWithPadding(tokenizer)
trainer = Trainer(model=model, tokenizer=tokenizer,
                  data_collator=data_collator,
                  args=training_args,
                  train_dataset=tokenized_ds["train"],
                  eval_dataset=tokenized_ds["test"], 
                  
                 )

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.205700,0.189076


TrainOutput(global_step=1563, training_loss=0.1978155451559212, metrics={'train_runtime': 431.6714, 'train_samples_per_second': 57.914, 'train_steps_per_second': 3.621, 'total_flos': 3284752688410752.0, 'train_loss': 0.1978155451559212, 'epoch': 1.0})